In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img
import matplotlib.pyplot as plt

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download_DCT_train = drive.CreateFile({'id': 'here put id '})  
download_DCT_train.GetContentFile('dataset.zip')

In [ ]:
!unzip /content/dataset.zip -d /content/data

In [ ]:
#train_datagen = dict(featurewise_center=True,
                     ##rescale=1./255,
                     #featurewise_std_normalization=True,
                     ##rotation_range=90,
                     #width_shift_range=0.1,
                     #height_shift_range=0.1,
                     ##zoom_range=0.2)
train_datagen = ImageDataGenerator(rescale=1/255,)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1

train_image_generator = train_datagen.flow_from_directory(
    '/content/data/training/images',
    target_size=(256, 256),
    color_mode='rgb',   
    batch_size=64,
    class_mode=None,     
    seed=seed)

train_mask_generator = train_datagen.flow_from_directory(
    '/content/data/training/masks',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=64,
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(train_image_generator, train_mask_generator) 

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1


valid_image_generator = valid_datagen.flow_from_directory(
    '/content/data/validation/images',
    target_size=(256, 256),
    color_mode='rgb',
    batch_size=32,
    class_mode=None,
    seed=seed)

valid_mask_generator = valid_datagen.flow_from_directory(
    '/content/data/validation/masks',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
validation_generator = zip(valid_image_generator, valid_mask_generator) 

In [ ]:
plt.imshow(train_image_generator[6][3])

In [ ]:
plt.imshow(array_to_img(train_mask_generator[6][3]))

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=np.ceil(1143/64), 
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=np.ceil(246/32))